In [2]:

#import required libraries
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
 
# load ascii text and covert to lowercase
filename = "/kaggle/input/ebooksentence/6761-0.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
 
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
 
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
 
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
 
# reshape X to be [samples, time steps, features]
X = torch.tensor(dataX, dtype=torch.float32).reshape(n_patterns, seq_length, 1)
X = X / float(n_vocab)
y = torch.tensor(dataY)
 
class CharModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=256, num_layers=2, batch_first=True, dropout=0.2)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(256, n_vocab)
    def forward(self, x):
        x, _ = self.lstm(x)
        # take only the last output
        x = x[:, -1, :]
        # produce output
        x = self.linear(self.dropout(x))
        return x

#set the hyperparameters
n_epochs = 40
batch_size = 128
model = CharModel()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
 
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss(reduction="sum")
loader = data.DataLoader(data.TensorDataset(X, y), shuffle=True, batch_size=batch_size)
 
best_model = None
best_loss = np.inf

#run the model for the given epochs
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch.to(device))
        loss = loss_fn(y_pred, y_batch.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    model.eval()
    loss = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            y_pred = model(X_batch.to(device))
            loss += loss_fn(y_pred, y_batch.to(device))
        if loss < best_loss:
            best_loss = loss
            best_model = model.state_dict()
        print("Epoch %d: Cross-entropy: %.4f" % (epoch, loss))
 
torch.save([best_model, char_to_int], "single-char.pth")
 
# Generation using the trained model
best_model, char_to_int = torch.load("single-char.pth")
n_vocab = len(char_to_int)
int_to_char = dict((i, c) for c, i in char_to_int.items())
model.load_state_dict(best_model)

Total Characters:  974238
Total Vocab:  62
Total Patterns:  974138
Epoch 0: Cross-entropy: 2131908.7500
Epoch 1: Cross-entropy: 1863718.6250
Epoch 2: Cross-entropy: 1710715.0000
Epoch 3: Cross-entropy: 1641318.6250
Epoch 4: Cross-entropy: 1549532.2500
Epoch 5: Cross-entropy: 1501205.8750
Epoch 6: Cross-entropy: 1471907.5000
Epoch 7: Cross-entropy: 1437843.2500
Epoch 8: Cross-entropy: 1409095.5000
Epoch 9: Cross-entropy: 1386524.6250
Epoch 10: Cross-entropy: 1391955.1250
Epoch 11: Cross-entropy: 1348702.3750
Epoch 12: Cross-entropy: 1337647.6250
Epoch 13: Cross-entropy: 1357728.5000
Epoch 14: Cross-entropy: 1317505.6250
Epoch 15: Cross-entropy: 1319120.3750
Epoch 16: Cross-entropy: 1303259.8750
Epoch 17: Cross-entropy: 1284446.3750
Epoch 18: Cross-entropy: 1273782.3750
Epoch 19: Cross-entropy: 1264423.1250
Epoch 20: Cross-entropy: 1265295.7500
Epoch 21: Cross-entropy: 1249367.2500
Epoch 22: Cross-entropy: 1245870.0000
Epoch 23: Cross-entropy: 1238497.5000
Epoch 24: Cross-entropy: 124169

<All keys matched successfully>

In [3]:
# randomly generate a prompt
filename = "/kaggle/input/ebooksentence/6761-0.txt"
seq_length = 100
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
start = np.random.randint(0, len(raw_text)-seq_length)
prompt = raw_text[start:start+seq_length]
pattern = [char_to_int[c] for c in prompt]  
 
model.eval()
print('Prompt: "%s"' % prompt)
with torch.no_grad():
    for i in range(1000):
        # format input array of int into PyTorch tensor
        x = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
        x = torch.tensor(x, dtype=torch.float32)
        # generate logits as output from the model
        prediction = model(x.to(device))
        # convert logits into one character
        index = int(prediction.argmax())
        result = int_to_char[index]
        print(result, end="")
        # append the new character into the prompt for the next iteration
        pattern.append(index)
        pattern = pattern[1:]
print()
print("Done.")

Prompt: "een cherished by the conversation of her school-fellows. she
was particularly fond of music, in whic"
h he was already seen to the project gutenberg-tm electronic work with the service of the service, and the count was not a language to the project gutenberg-tm electronic work and despair, and the count was not a language to the project gutenberg-tm electronic work and despair, and the count was not a language to the project gutenberg-tm electronic work and despair, and the count was not a language to the project gutenberg-tm electronic work and despair, and the count was not a language to the project gutenberg-tm electronic work and despair, and the count was not a language to the project gutenberg-tm electronic work and despair, and the count was not a language to the project gutenberg-tm electronic work and despair, and the count was not a language to the project gutenberg-tm electronic work and despair, and the count was not a language to the project gutenberg-tm electroni

True